In [2]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc
from imblearn.over_sampling import SMOTE
import shap

import matplotlib.pyplot as plt

In [8]:
# read raw data
def data_loader(file_name):
    current_dir = os.getcwd()
    notebook_folder = os.path.dirname(current_dir)
    project_dir = os.path.dirname(notebook_folder)
    file_path = os.path.join(project_dir, 'raw_data', file_name)
    return pd.read_csv(file_path)

raw_data = data_loader('raw.csv')
print(raw_data.shape)
print(raw_data.head())

(50801, 77)
   delist_tab  month_count  _201901  _201902  _201903  _201904   _201905  \
0           0           42  26994.0   8180.0  16360.0  26994.0  39264.00   
1           0            5      NaN      NaN      NaN      NaN       NaN   
2           0           46    330.0    220.0    220.0    477.0    275.00   
3           0           53   7636.5    820.0   2738.0   4243.0   7545.35   
4           0            8    742.5    148.5      NaN    742.5       NaN   

    _201911    _201912   _202007  ...  max_city  min_city  median_city  \
0   91152.0  1002672.0  273447.0  ...         6         1          3.0   
1  182400.0    91200.0       NaN  ...         1         1          1.0   
2     643.6      642.0     440.0  ...         4         1          2.0   
3    4931.0     3085.0    1064.0  ...         6         1          3.0   
4      54.0        NaN       NaN  ...         1         1          1.0   

   city_now  vbp_flag  VBP_Batch  VBP_time  NRDL  NRDL_LIMTT  ENTRY_TIME  
0       3.0

In [ ]:
# preprocess data 
def preprocess_data(data, target_var_name, categorical_var_names, scaling_method):
    # Separate target variable and input variables
    y = data[[target_var_name]]
    X = data.drop(columns=[target_var_name])

    # Separate numerical and categorical variables
    numerical_vars = X.columns.difference(categorical_var_names)
    
    X_numerical = X[numerical_vars]
    X_categorical = X[categorical_var_names]

    # Handle missing values for numerical variables
    X_numerical.fillna(0, inplace=True)

    # Scale numerical variables
    if scaling_method == 'MinMaxScaler':
        scaler = MinMaxScaler()
        X_numerical = pd.DataFrame(scaler.fit_transform(X_numerical), columns=numerical_vars)

    # Encode categorical variables
    if not X_categorical.empty:
        for col in categorical_var_names:
            X_categorical[col] = LabelEncoder().fit_transform(X_categorical[col])

        X_categorical = pd.get_dummies(X_categorical, columns=categorical_var_names, drop_first=True, dtype=int)

    # Concatenate numerical and categorical variables
    X = pd.concat([X_numerical, X_categorical], axis=1)
    
    return X, y


In [ ]:

def save_processed_data(data, filename):
    output_filepath = os.path.join('processed', filename + '.csv')
    data.to_csv(output_filepath, index=False)


def data_loader(file_name):
    current_dir = os.getcwd()
    notebook_folder = os.path.dirname(current_dir)
    project_dir = os.path.dirname(notebook_folder)
    file_path = os.path.join(project_dir, 'raw_data', file_name)
    return pd.read_csv(file_path)